In [41]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from utils import PkdbModel, caffeine_idx, pktype_data, filter_out, convert_unit, unstring
plt.style.use('seaborn-whitegrid')
import numpy as np

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
def to_numeric(df_data):
    df_data[("weight","value")] = pd.to_numeric(df_data[("weight","value")])
    df_data[("weight","mean")] = pd.to_numeric(df_data[("weight","mean")])
    df_data["value_intervention"] = pd.to_numeric(df_data["value_intervention"])
    df_data["median"] = pd.to_numeric(df_data["median"])
    df_data["mean"] = pd.to_numeric(df_data["mean"])
    df_data["value"] = pd.to_numeric(df_data["value"])
    df_data["se"] = pd.to_numeric(df_data["se"])
    df_data["sd"] = pd.to_numeric(df_data["sd"])
    df_data["cv"] = pd.to_numeric(df_data["cv"])
    return df_data
     
def infer_from_interventions(data, unit_abs = "gram"):
    unit_rel= unit_abs + " / kilogram"
    data_rel = convert_unit(data,
                             unit_in=unit_abs,
                             unit_out=unit_rel, 
                             factor=1.0/data[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","value"),"value"])

    data_abs = convert_unit(data,
                             unit_in=unit_rel,
                             unit_out=unit_abs, 
                             factor=data[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","value"),"value"])
    
    data = pd.concat([data,data_rel,data_abs], ignore_index=True)
    
    data_rel = convert_unit(data,
                             unit_in=unit_abs,
                             unit_out=unit_rel, 
                             factor=1.0/data[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","mean"),"mean"])

    data_abs = convert_unit(data,
                             unit_in=unit_rel,
                             unit_out=unit_abs, 
                             factor=data[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","mean"),"mean"])
    
    data = pd.concat([data,data_rel,data_abs], ignore_index=True)
    
    return data

def infer_from_outputs(data, unit_abs="liter / hour"):
    unit_rel= unit_abs + " / kilogram"
    data_rel = convert_unit(data,
                                 unit_in=unit_abs,
                                 unit_out=unit_rel, 
                                 factor=1.0/data[("weight","value")],
                                 unit_field="unit", 
                                 data_fields=['value'], 
                                 subset=[("weight","value"),"value"])

    data_abs_i = convert_unit(data,
                                 unit_in=unit_rel,
                                 unit_out=unit_abs, 
                                 factor=data[("weight","value")],
                                 unit_field="unit", 
                                 data_fields=['value'],
                                 subset=[("weight","value"),"value"])
    

    data = pd.concat([data,data_rel,data_abs_i], ignore_index=True)
    
    data_rel = convert_unit(data,
                             unit_in=unit_abs,
                             unit_out=unit_rel, 
                             factor=1.0/data[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])

    data_abs = convert_unit(data,
                             unit_in=unit_rel,
                             unit_out=unit_abs, 
                             factor=data[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])


    data = pd.concat([data,data_rel,data_abs], ignore_index=True)
    
    return data

In [44]:
all_complete = PkdbModel("all_complete", destination="2-merged")
all_complete.read()
all_complete.data.loc[(all_complete.data[("sex","choice")] == "M"),("oral contraceptives","choice")] = "N"
complete_caffeine = all_complete.data[caffeine_idx(all_complete.data)]

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (20,23,24,25,35,36,80,86,98,100,102,104,107,111,115,120,121,122,123,126,134,135,136,144,153,154,158,161,162,164,165,166,167,176,179,180,181,189,196,197,204,209,214,219,224,232,238,239,246,252,258,268) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [45]:
caffeine_clearance = pktype_data(complete_caffeine,"clearance")
caffeine_clearance = filter_out(caffeine_clearance,"unit",["milliliter / meter ** 2 / minute"])

In [46]:
caffeine_clearance.groupby(["unit"]).apply(len)

unit
liter / hour               404
liter / hour / kilogram    389
dtype: int64

In [47]:
caffeine_clearance = to_numeric(caffeine_clearance)
caffeine_clearance = infer_from_interventions(caffeine_clearance,"gram")
caffeine_clearance = infer_from_outputs(caffeine_clearance,"liter / hour")

In [48]:
caffeine_clr = PkdbModel("caffeine_clearance",destination="3-inferred")
caffeine_clr.data = caffeine_clearance
caffeine_clr.save()

In [49]:
caffeine_clearance.groupby(["unit"]).apply(len)

unit
liter / hour               1071
liter / hour / kilogram    1075
dtype: int64

## Timecourses

In [50]:
caffeine_timecourse = pktype_data(complete_caffeine,"concentration")
caffeine_timecourse = filter_out(caffeine_timecourse,"output_type",["outputs"])


In [51]:
caffeine_timecourse.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  raw_pk  tissue  \
unit         unit_intervention                                            
gram / liter gram                 103          103  103     103     103   
             gram / kilogram        9            9    9       9       9   
mole / liter gram                   5            5    5       5       5   
             gram / kilogram       12           12   12      12      12   

                                interventions  substance_name  pktype  \
unit         unit_intervention                                          
gram / liter gram                         103             103     103   
             gram / kilogram                9               9       9   
mole / liter gram                           5               5       5   
             gram / kilogram               12              12      12   

                                calculated  individual_pk  ...  \
unit         unit_intervention                             ...   
gram / liter gram                        0             86  ...   
             gram / kilogram             0              5  ...   
mole / liter gram                        0              0  ...   
             gram / kilogram             0              0  ...   

                                (metabolic ratio, count)  \
unit         unit_intervention                             
gram / liter gram                                      0   
             gram / kilogram                           0   
mole / liter gram                                      0   
             gram / kilogram                           0   

                                (metabolic ratio, max)  \
unit         unit_intervention                           
gram / liter gram                                    0   
             gram / kilogram                         0   
mole / liter gram                                    0   
             gram / kilogram                         0   

                                (metabolic ratio, mean)  \
unit         unit_intervention                            
gram / liter gram                                     0   
             gram / kilogram                          0   
mole / liter gram                                     0   
             gram / kilogram                          0   

                                (metabolic ratio, min)  \
unit         unit_intervention                           
gram / liter gram                                    0   
             gram / kilogram                         0   
mole / liter gram                                    0   
             gram / kilogram                         0   

                                (metabolic ratio, unit)  (fat weight, count)  \
unit         unit_intervention                                                 
gram / liter gram                                     0                    0   
             gram / kilogram                          0                    0   
mole / liter gram                                     0                    0   
             gram / kilogram                          0                   12   

                                (fat weight, cv)  (fat weight, mean)  \
unit         unit_intervention                                         
gram / liter gram                              0                   0   
             gram / kilogram                   0                   0   
mole / liter gram                              0                   0   
             gram / kilogram                  12                  12   

                                (fat weight, se)  (fat weight, unit)  
unit         unit_intervention                                        
gram / liter gram                              0                   0  
             gram / kilogram                   0                   0  
mole / liter gram                              0                   0  
             gram / kilogram                  12              

In [52]:
caffeine_timecourse = filter_out(caffeine_timecourse,"unit",["mole / liter"])
caffeine_timecourse.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  raw_pk  tissue  \
unit         unit_intervention                                            
gram / liter gram                 103          103  103     103     103   
             gram / kilogram        9            9    9       9       9   

                                interventions  substance_name  pktype  \
unit         unit_intervention                                          
gram / liter gram                         103             103     103   
             gram / kilogram                9               9       9   

                                calculated  individual_pk  ...  \
unit         unit_intervention                             ...   
gram / liter gram                        0             86  ...   
             gram / kilogram             0              5  ...   

                                (metabolic ratio, count)  \
unit         unit_intervention                             
gram / liter gram                                      0   
             gram / kilogram                           0   

                                (metabolic ratio, max)  \
unit         unit_intervention                           
gram / liter gram                                    0   
             gram / kilogram                         0   

                                (metabolic ratio, mean)  \
unit         unit_intervention                            
gram / liter gram                                     0   
             gram / kilogram                          0   

                                (metabolic ratio, min)  \
unit         unit_intervention                           
gram / liter gram                                    0   
             gram / kilogram                         0   

                                (metabolic ratio, unit)  (fat weight, count)  \
unit         unit_intervention                                                 
gram / liter gram                                     0                    0   
             gram / kilogram                          0                    0   

                                (fat weight, cv)  (fat weight, mean)  \
unit         unit_intervention                                         
gram / liter gram                              0                   0   
             gram / kilogram                   0                   0   

                                (fat weight, se)  (fat weight, unit)  
unit         unit_intervention                                        
gram / liter gram                              0                   0  
             gram / kilogram                   0                   0  

[2 rows x 266 columns]

In [53]:
caffeine_tc = PkdbModel("caffeine_timecourse",destination="3-inferred")
caffeine_tc.data = caffeine_timecourse
caffeine_tc.to_array()
caffeine_timecourse = caffeine_tc.data

In [54]:
caffeine_timecourse = infer_from_interventions(caffeine_timecourse,"gram")
caffeine_timecourse = infer_from_outputs(caffeine_timecourse,"gram / liter")

In [55]:
caffeine_timecourse.groupby(["unit","unit_intervention","time_unit"]).count()

study  output_type   pk  \
unit                    unit_intervention time_unit                            
gram / liter            gram              hr           105          105  105   
                        gram / kilogram   hr           102          102  102   
gram / liter / kilogram gram              hr            95           95   95   
                        gram / kilogram   hr            95           95   95   

                                                     raw_pk  tissue  \
unit                    unit_intervention time_unit                   
gram / liter            gram              hr            105     105   
                        gram / kilogram   hr            102     102   
gram / liter / kilogram gram              hr             95      95   
                        gram / kilogram   hr             95      95   

                                                     interventions  \
unit                    unit_intervention time_unit                  
gram / liter            gram              hr                   105   
                        gram / kilogram   hr                   102   
gram / liter / kilogram gram              hr                    95   
                        gram / kilogram   hr                    95   

                                                     substance_name  pktype  \
unit                    unit_intervention time_unit                           
gram / liter            gram              hr                    105     105   
                        gram / kilogram   hr                    102     102   
gram / liter / kilogram gram              hr                     95      95   
                        gram / kilogram   hr                     95      95   

                                                     calculated  \
unit                    unit_intervention time_unit               
gram / liter            gram              hr                  0   
                        gram / kilogram   hr                  0   
gram / liter / kilogram gram              hr                  0   
                        gram / kilogram   hr                  0   

                                                     individual_pk  ...  \
unit                    unit_intervention time_unit                 ...   
gram / liter            gram              hr                    86  ...   
                        gram / kilogram   hr                    89  ...   
gram / liter / kilogram gram              hr                    84  ...   
                        gram / kilogram   hr                    84  ...   

                                                     (metabolic ratio, count)  \
unit                    unit_intervention time_unit                             
gram / liter            gram              hr                                0   
                        gram / kilogram   hr                                0   
gram / liter / kilogram gram              hr                                0   
                        gram / kilogram   hr                                0   

                                                     (metabolic ratio, max)  \
unit                    unit_intervention time_unit                           
gram / liter            gram              hr                              0   
                        gram / kilogram   hr                              0   
gram / liter / kilogram gram              hr                              0   
                        gram / kilogram   hr                              0   

                                                     (metabolic ratio, mean)  \
unit                    unit_intervention time_unit                            
gram / liter            gram              hr                               0   
                        gram / kilogram   hr                               0   
gram / liter / kilogram gram              hr                               0   
                        gram / ki

In [56]:
caffeine_tc = PkdbModel("caffeine_timecourse",destination="3-inferred")
caffeine_tc.data = caffeine_timecourse
caffeine_tc.save()

In [57]:
caffeine_thalf = pktype_data(complete_caffeine,"thalf")
caffeine_thalf = to_numeric(caffeine_thalf)
caffeine_thalf.groupby(["unit","unit_intervention"]).count()


/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

study  output_type   pk  raw_pk  tissue  \
unit unit_intervention                                            
hour gram                 512          512  512     512     512   
     gram / kilogram       79           79   79      79      79   

                        interventions  substance_name  pktype  calculated  \
unit unit_intervention                                                      
hour gram                         512             512     512         512   
     gram / kilogram               79              79      79          79   

                        individual_pk  ...  (metabolic ratio, count)  \
unit unit_intervention                 ...                             
hour gram                         429  ...                         0   
     gram / kilogram               41  ...                         0   

                        (metabolic ratio, max)  (metabolic ratio, mean)  \
unit unit_intervention                                                    
hour gram                                    0                        0   
     gram / kilogram                         0                        0   

                        (metabolic ratio, min)  (metabolic ratio, unit)  \
unit unit_intervention                                                    
hour gram                                    0                        0   
     gram / kilogram                         0                        0   

                        (fat weight, count)  (fat weight, cv)  \
unit unit_intervention                                          
hour gram                                 0                 0   
     gram / kilogram                     18                18   

                        (fat weight, mean)  (fat weight, se)  \
unit unit_intervention                                         
hour gram                                0                 0   
     gram / kilogram                    18                18   

                        (fat weight, unit)  
unit unit_intervention                      
hour gram                                0  
     gram / kilogram                    18  

[2 rows x 266 columns]

In [58]:
caffeine_thalf = infer_from_interventions(caffeine_thalf,"gram")
caffeine_thalf = infer_from_outputs(caffeine_thalf,"hour")

In [59]:
caffeine_thalf.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  raw_pk  tissue  \
unit            unit_intervention                                            
hour            gram                 570          570  570     570     570   
                gram / kilogram      461          461  461     461     461   
hour / kilogram gram                 440          440  440     440     440   
                gram / kilogram      440          440  440     440     440   

                                   interventions  substance_name  pktype  \
unit            unit_intervention                                          
hour            gram                         570             570     570   
                gram / kilogram              461             461     461   
hour / kilogram gram                         440             440     440   
                gram / kilogram              440             440     440   

                                   calculated  individual_pk  ...  \
unit            unit_intervention                             ...   
hour            gram                      570            453  ...   
                gram / kilogram           461            375  ...   
hour / kilogram gram                      440            358  ...   
                gram / kilogram           440            358  ...   

                                   (metabolic ratio, count)  \
unit            unit_intervention                             
hour            gram                                      0   
                gram / kilogram                           0   
hour / kilogram gram                                      0   
                gram / kilogram                           0   

                                   (metabolic ratio, max)  \
unit            unit_intervention                           
hour            gram                                    0   
                gram / kilogram                         0   
hour / kilogram gram                                    0   
                gram / kilogram                         0   

                                   (metabolic ratio, mean)  \
unit            unit_intervention                            
hour            gram                                     0   
                gram / kilogram                          0   
hour / kilogram gram                                     0   
                gram / kilogram                          0   

                                   (metabolic ratio, min)  \
unit            unit_intervention                           
hour            gram                                    0   
                gram / kilogram                         0   
hour / kilogram gram                                    0   
                gram / kilogram                         0   

                                   (metabolic ratio, unit)  \
unit            unit_intervention                            
hour            gram                                     0   
                gram / kilogram                          0   
hour / kilogram gram                                     0   
                gram / kilogram                          0   

                                   (fat weight, count)  (fat weight, cv)  \
unit            unit_intervention                                          
hour            gram                                18                18   
                gram / kilogram                     18                18   
hour / kilogram gram                                18                18   
                gram / kilogram                     18                18   

                                   (fat weight, mean)  (fat weight, se)  \
unit            unit_intervention                                         
hour            gram                               18                18   
                gram / kilogram                    18                18   
hour / kilogram gram                               18                18   
   

In [60]:
caffeine_thf = PkdbModel("caffeine_thalf",destination="3-inferred")
caffeine_thf.data = caffeine_thalf
caffeine_thf.save()

In [61]:
caffeine_tmax = pktype_data(complete_caffeine,"tmax")
caffeine_tmax = to_numeric(caffeine_tmax)

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [62]:
caffeine_tmax.groupby(["unit","unit_intervention"]).count()


study  output_type   pk  raw_pk  tissue  \
unit unit_intervention                                            
hour gram                 114          114  114     114     114   
     gram / kilogram       23           23   23      23      23   

                        interventions  substance_name  pktype  calculated  \
unit unit_intervention                                                      
hour gram                         114             114     114         114   
     gram / kilogram               23              23      23          23   

                        individual_pk  ...  (metabolic ratio, count)  \
unit unit_intervention                 ...                             
hour gram                          77  ...                         0   
     gram / kilogram               12  ...                         0   

                        (metabolic ratio, max)  (metabolic ratio, mean)  \
unit unit_intervention                                                    
hour gram                                    0                        0   
     gram / kilogram                         0                        0   

                        (metabolic ratio, min)  (metabolic ratio, unit)  \
unit unit_intervention                                                    
hour gram                                    0                        0   
     gram / kilogram                         0                        0   

                        (fat weight, count)  (fat weight, cv)  \
unit unit_intervention                                          
hour gram                                 0                 0   
     gram / kilogram                      8                 8   

                        (fat weight, mean)  (fat weight, se)  \
unit unit_intervention                                         
hour gram                                0                 0   
     gram / kilogram                     8                 8   

                        (fat weight, unit)  
unit unit_intervention                      
hour gram                                0  
     gram / kilogram                     8  

[2 rows x 266 columns]

In [63]:
caffeine_tmax = infer_from_interventions(caffeine_tmax)
caffeine_tmax = infer_from_outputs(caffeine_tmax, "hour")

In [64]:
caffeine_tmax.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  raw_pk  tissue  \
unit            unit_intervention                                            
hour            gram                 123          123  123     123     123   
                gram / kilogram       66           66   66      66      66   
hour / kilogram gram                  52           52   52      52      52   
                gram / kilogram       52           52   52      52      52   

                                   interventions  substance_name  pktype  \
unit            unit_intervention                                          
hour            gram                         123             123     123   
                gram / kilogram               66              66      66   
hour / kilogram gram                          52              52      52   
                gram / kilogram               52              52      52   

                                   calculated  individual_pk  ...  \
unit            unit_intervention                             ...   
hour            gram                      123             77  ...   
                gram / kilogram            66             32  ...   
hour / kilogram gram                       52             20  ...   
                gram / kilogram            52             20  ...   

                                   (metabolic ratio, count)  \
unit            unit_intervention                             
hour            gram                                      0   
                gram / kilogram                           0   
hour / kilogram gram                                      0   
                gram / kilogram                           0   

                                   (metabolic ratio, max)  \
unit            unit_intervention                           
hour            gram                                    0   
                gram / kilogram                         0   
hour / kilogram gram                                    0   
                gram / kilogram                         0   

                                   (metabolic ratio, mean)  \
unit            unit_intervention                            
hour            gram                                     0   
                gram / kilogram                          0   
hour / kilogram gram                                     0   
                gram / kilogram                          0   

                                   (metabolic ratio, min)  \
unit            unit_intervention                           
hour            gram                                    0   
                gram / kilogram                         0   
hour / kilogram gram                                    0   
                gram / kilogram                         0   

                                   (metabolic ratio, unit)  \
unit            unit_intervention                            
hour            gram                                     0   
                gram / kilogram                          0   
hour / kilogram gram                                     0   
                gram / kilogram                          0   

                                   (fat weight, count)  (fat weight, cv)  \
unit            unit_intervention                                          
hour            gram                                 8                 8   
                gram / kilogram                      8                 8   
hour / kilogram gram                                 8                 8   
                gram / kilogram                      8                 8   

                                   (fat weight, mean)  (fat weight, se)  \
unit            unit_intervention                                         
hour            gram                                8                 8   
                gram / kilogram                     8                 8   
hour / kilogram gram                                8                 8   
   

In [65]:
caffeine_tmx = PkdbModel("caffeine_tmax",destination="3-inferred")
caffeine_tmx.data = caffeine_tmax
caffeine_tmx.save()

In [66]:
caffeine_vd = pktype_data(complete_caffeine,"vd")
caffeine_vd = to_numeric(caffeine_vd)

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [67]:
caffeine_vd.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  raw_pk  tissue  \
unit             unit_intervention                                            
liter            gram                 242          242  242     242     242   
                 gram / kilogram       11           11   11      11      11   
liter / kilogram gram                 160          160  160     160     160   
                 gram / kilogram       56           56   56      56      56   

                                    interventions  substance_name  pktype  \
unit             unit_intervention                                          
liter            gram                         242             242     242   
                 gram / kilogram               11              11      11   
liter / kilogram gram                         160             160     160   
                 gram / kilogram               56              56      56   

                                    calculated  individual_pk  ...  \
unit             unit_intervention                             ...   
liter            gram                      242            207  ...   
                 gram / kilogram            11              0  ...   
liter / kilogram gram                      160            120  ...   
                 gram / kilogram            56             26  ...   

                                    (metabolic ratio, count)  \
unit             unit_intervention                             
liter            gram                                      0   
                 gram / kilogram                           0   
liter / kilogram gram                                      0   
                 gram / kilogram                           0   

                                    (metabolic ratio, max)  \
unit             unit_intervention                           
liter            gram                                    0   
                 gram / kilogram                         0   
liter / kilogram gram                                    0   
                 gram / kilogram                         0   

                                    (metabolic ratio, mean)  \
unit             unit_intervention                            
liter            gram                                     0   
                 gram / kilogram                          0   
liter / kilogram gram                                     0   
                 gram / kilogram                          0   

                                    (metabolic ratio, min)  \
unit             unit_intervention                           
liter            gram                                    0   
                 gram / kilogram                         0   
liter / kilogram gram                                    0   
                 gram / kilogram                         0   

                                    (metabolic ratio, unit)  \
unit             unit_intervention                            
liter            gram                                     0   
                 gram / kilogram                          0   
liter / kilogram gram                                     0   
                 gram / kilogram                          0   

                                    (fat weight, count)  (fat weight, cv)  \
unit             unit_intervention                                          
liter            gram                                 0                 0   
                 gram / kilogram                      0                 0   
liter / kilogram gram                                 0                 0   
                 gram / kilogram                     18                18   

                                    (fat weight, mean)  (fat weight, se)  \
unit             unit_intervention                                         
liter            gram                                0                 0   
                 gram / kilogram                     0                 0   
liter / kilogram gram

In [68]:
caffeine_vd =  infer_from_interventions(caffeine_vd, "gram")
caffeine_vd =  infer_from_outputs(caffeine_vd, "liter")

In [69]:
caffeine_vd.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  raw_pk  tissue  \
unit             unit_intervention                                            
liter            gram                 381          381  381     381     381   
                 gram / kilogram      337          337  337     337     337   
liter / kilogram gram                 404          404  404     404     404   
                 gram / kilogram      358          358  358     358     358   

                                    interventions  substance_name  pktype  \
unit             unit_intervention                                          
liter            gram                         381             381     381   
                 gram / kilogram              337             337     337   
liter / kilogram gram                         404             404     404   
                 gram / kilogram              358             358     358   

                                    calculated  individual_pk  ...  \
unit             unit_intervention                             ...   
liter            gram                      381            282  ...   
                 gram / kilogram           337            261  ...   
liter / kilogram gram                      404            315  ...   
                 gram / kilogram           358            278  ...   

                                    (metabolic ratio, count)  \
unit             unit_intervention                             
liter            gram                                      0   
                 gram / kilogram                           0   
liter / kilogram gram                                      0   
                 gram / kilogram                           0   

                                    (metabolic ratio, max)  \
unit             unit_intervention                           
liter            gram                                    0   
                 gram / kilogram                         0   
liter / kilogram gram                                    0   
                 gram / kilogram                         0   

                                    (metabolic ratio, mean)  \
unit             unit_intervention                            
liter            gram                                     0   
                 gram / kilogram                          0   
liter / kilogram gram                                     0   
                 gram / kilogram                          0   

                                    (metabolic ratio, min)  \
unit             unit_intervention                           
liter            gram                                    0   
                 gram / kilogram                         0   
liter / kilogram gram                                    0   
                 gram / kilogram                         0   

                                    (metabolic ratio, unit)  \
unit             unit_intervention                            
liter            gram                                     0   
                 gram / kilogram                          0   
liter / kilogram gram                                     0   
                 gram / kilogram                          0   

                                    (fat weight, count)  (fat weight, cv)  \
unit             unit_intervention                                          
liter            gram                                18                18   
                 gram / kilogram                     18                18   
liter / kilogram gram                                18                18   
                 gram / kilogram                     18                18   

                                    (fat weight, mean)  (fat weight, se)  \
unit             unit_intervention                                         
liter            gram                               18                18   
                 gram / kilogram                    18                18   
liter / kilogram gram

In [70]:
caffeine_vd_model = PkdbModel("caffeine_vd",destination="3-inferred")
caffeine_vd_model.data = caffeine_vd
caffeine_vd_model.save()

In [71]:
caffeine_auc_inf = pktype_data(complete_caffeine,"auc_inf")
caffeine_auc_inf = to_numeric(caffeine_auc_inf)
caffeine_auc_inf.groupby(["unit","unit_intervention"]).count()

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

study  output_type   pk  raw_pk  \
unit                unit_intervention                                    
gram * hour / liter gram                 317          317  317     317   
                    gram / kilogram       29           29   29      29   

                                       tissue  interventions  substance_name  \
unit                unit_intervention                                          
gram * hour / liter gram                  317            317             317   
                    gram / kilogram        29             29              29   

                                       pktype  calculated  individual_pk  ...  \
unit                unit_intervention                                     ...   
gram * hour / liter gram                  317         317            280  ...   
                    gram / kilogram        29          29              5  ...   

                                       (metabolic ratio, count)  \
unit                unit_intervention                             
gram * hour / liter gram                                      0   
                    gram / kilogram                           0   

                                       (metabolic ratio, max)  \
unit                unit_intervention                           
gram * hour / liter gram                                    0   
                    gram / kilogram                         0   

                                       (metabolic ratio, mean)  \
unit                unit_intervention                            
gram * hour / liter gram                                     0   
                    gram / kilogram                          0   

                                       (metabolic ratio, min)  \
unit                unit_intervention                           
gram * hour / liter gram                                    0   
                    gram / kilogram                         0   

                                       (metabolic ratio, unit)  \
unit                unit_intervention                            
gram * hour / liter gram                                     0   
                    gram / kilogram                          0   

                                       (fat weight, count)  (fat weight, cv)  \
unit                unit_intervention                                          
gram * hour / liter gram                                 0                 0   
                    gram / kilogram                     18                18   

                                       (fat weight, mean)  (fat weight, se)  \
unit                unit_intervention                                         
gram * hour / liter gram                                0                 0   
                    gram / kilogram                    18                18   

                                       (fat weight, unit)  
unit                unit_intervention                      
gram * hour / liter gram                                0  
                    gram / kilogram                    18  

[2 rows x 266 columns]

In [72]:
caffeine_auc_inf =  infer_from_interventions(caffeine_auc_inf, "gram ")
caffeine_auc_inf =  infer_from_outputs(caffeine_auc_inf, "gram * hour / liter")

In [73]:
caffeine_auc_inf.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  \
unit                           unit_intervention                            
gram * hour / liter            gram                 317          317  317   
                               gram / kilogram       29           29   29   
gram * hour / liter / kilogram gram                 271          271  271   
                               gram / kilogram       22           22   22   

                                                  raw_pk  tissue  \
unit                           unit_intervention                   
gram * hour / liter            gram                  317     317   
                               gram / kilogram        29      29   
gram * hour / liter / kilogram gram                  271     271   
                               gram / kilogram        22      22   

                                                  interventions  \
unit                           unit_intervention                  
gram * hour / liter            gram                         317   
                               gram / kilogram               29   
gram * hour / liter / kilogram gram                         271   
                               gram / kilogram               22   

                                                  substance_name  pktype  \
unit                           unit_intervention                           
gram * hour / liter            gram                          317     317   
                               gram / kilogram                29      29   
gram * hour / liter / kilogram gram                          271     271   
                               gram / kilogram                22      22   

                                                  calculated  individual_pk  \
unit                           unit_intervention                              
gram * hour / liter            gram                      317            280   
                               gram / kilogram            29              5   
gram * hour / liter / kilogram gram                      271            244   
                               gram / kilogram            22              0   

                                                  ...  \
unit                           unit_intervention  ...   
gram * hour / liter            gram               ...   
                               gram / kilogram    ...   
gram * hour / liter / kilogram gram               ...   
                               gram / kilogram    ...   

                                                  (metabolic ratio, count)  \
unit                           unit_intervention                             
gram * hour / liter            gram                                      0   
                               gram / kilogram                           0   
gram * hour / liter / kilogram gram                                      0   
                               gram / kilogram                           0   

                                                  (metabolic ratio, max)  \
unit                           unit_intervention                           
gram * hour / liter            gram                                    0   
                               gram / kilogram                         0   
gram * hour / liter / kilogram gram                                    0   
                               gram / kilogram                         0   

                                                  (metabolic ratio, mean)  \
unit                           unit_intervention                            
gram * hour / liter            gram                                     0   
                               gram / kilogram                          0   
gram * hour / liter / kilogram gram                                     0   
                               gram / kilogram                          0   

                                                  (metabolic ratio, min)  \
unit                           unit

In [74]:
caffeine_auc_inf_model = PkdbModel("caffeine_auc_inf",destination="3-inferred")
caffeine_auc_inf_model.data = caffeine_auc_inf
caffeine_auc_inf_model.save()

In [75]:
complete_caffeine.groupby("pktype").count()

,study,output_type,pk,raw_pk,tissue,interventions,unit,substance_name,calculated,individual_pk,...,"(metabolic ratio, count)","(metabolic ratio, max)","(metabolic ratio, mean)","(metabolic ratio, min)","(metabolic ratio, unit)","(fat weight, count)","(fat weight, cv)","(fat weight, mean)","(fat weight, se)","(fat weight, unit)"
pktype,,,,,,,,,,,,,,,,,,,,,
auc_end,156,156,156,156,156,156,156,156,156,109,...,0,0,0,0,0,10,10,10,10,10
auc_inf,346,346,346,346,346,346,346,346,346,285,...,0,0,0,0,0,18,18,18,18,18
clearance,795,795,795,795,795,795,795,795,795,656,...,0,0,0,0,0,18,18,18,18,18
clearance_unbound,5,5,5,5,5,5,5,5,5,0,...,0,0,0,0,0,0,0,0,0,0
cmax,264,264,264,264,264,264,264,264,264,178,...,0,0,0,0,0,18,18,18,18,18
concentration,181,181,181,181,181,181,181,181,52,131,...,0,0,0,0,0,12,12,12,12,12
fraction_absorbed,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
fraction_unbound,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
kabs,5,5,5,5,5,5,5,5,5,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
caffeine_kel = pktype_data(complete_caffeine,"kel")
caffeine_kel = to_numeric(caffeine_kel)

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [77]:
caffeine_kel.groupby(["unit","unit_intervention"]).count()


study  output_type   pk  raw_pk  tissue  \
unit       unit_intervention                                            
1 / minute gram                 212          212  212     212     212   
           gram / kilogram       52           52   52      52      52   

                              interventions  substance_name  pktype  \
unit       unit_intervention                                          
1 / minute gram                         212             212     212   
           gram / kilogram               52              52      52   

                              calculated  individual_pk  ...  \
unit       unit_intervention                             ...   
1 / minute gram                      212            186  ...   
           gram / kilogram            52             20  ...   

                              (metabolic ratio, count)  \
unit       unit_intervention                             
1 / minute gram                                      0   
           gram / kilogram                           0   

                              (metabolic ratio, max)  (metabolic ratio, mean)  \
unit       unit_intervention                                                    
1 / minute gram                                    0                        0   
           gram / kilogram                         0                        0   

                              (metabolic ratio, min)  (metabolic ratio, unit)  \
unit       unit_intervention                                                    
1 / minute gram                                    0                        0   
           gram / kilogram                         0                        0   

                              (fat weight, count)  (fat weight, cv)  \
unit       unit_intervention                                          
1 / minute gram                                 0                 0   
           gram / kilogram                     18                18   

                              (fat weight, mean)  (fat weight, se)  \
unit       unit_intervention                                         
1 / minute gram                                0                 0   
           gram / kilogram                    18                18   

                              (fat weight, unit)  
unit       unit_intervention                      
1 / minute gram                                0  
           gram / kilogram                    18  

[2 rows x 266 columns]

In [78]:
caffeine_kel =  infer_from_interventions(caffeine_kel, "gram")
caffeine_kel =  infer_from_outputs(caffeine_kel, "1/minute")

In [79]:
caffeine_kel.groupby(["unit","unit_intervention"]).count()


study  output_type   pk  raw_pk  tissue  \
unit       unit_intervention                                            
1 / minute gram                 257          257  257     257     257   
           gram / kilogram      255          255  255     255     255   

                              interventions  substance_name  pktype  \
unit       unit_intervention                                          
1 / minute gram                         257             257     257   
           gram / kilogram              255             255     255   

                              calculated  individual_pk  ...  \
unit       unit_intervention                             ...   
1 / minute gram                      257            201  ...   
           gram / kilogram           255            206  ...   

                              (metabolic ratio, count)  \
unit       unit_intervention                             
1 / minute gram                                      0   
           gram / kilogram                           0   

                              (metabolic ratio, max)  (metabolic ratio, mean)  \
unit       unit_intervention                                                    
1 / minute gram                                    0                        0   
           gram / kilogram                         0                        0   

                              (metabolic ratio, min)  (metabolic ratio, unit)  \
unit       unit_intervention                                                    
1 / minute gram                                    0                        0   
           gram / kilogram                         0                        0   

                              (fat weight, count)  (fat weight, cv)  \
unit       unit_intervention                                          
1 / minute gram                                18                18   
           gram / kilogram                     18                18   

                              (fat weight, mean)  (fat weight, se)  \
unit       unit_intervention                                         
1 / minute gram                               18                18   
           gram / kilogram                    18                18   

                              (fat weight, unit)  
unit       unit_intervention                      
1 / minute gram                               18  
           gram / kilogram                    18  

[2 rows x 266 columns]

In [80]:
caffeine_kel_model = PkdbModel("caffeine_kel",destination="3-inferred")
caffeine_kel_model.data = caffeine_kel
caffeine_kel_model.save()